### Tilt-series Alignment (IMOD)

In [ ]:
stats_yaml_file = f"{proj_name}_imod_align_stats.yaml"

with open(stats_yaml_file, "r") as f:
    stats = yaml.load(f.read(), Loader=yaml.FullLoader)
    
stats_df = pd.DataFrame.from_dict(stats).drop(columns=["index"])
stats_df["Tilt series"] = stats_df["Tilt series"].astype(int, copy=False)
# pd.set_option("display.max_rows", None)
display(stats_df)

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.barplot(data=stats_df,
                 x="Tilt series",
                 y="Error mean (nm)",
                 color="cornflowerblue"
                )

ax.errorbar(x=stats_df.index,
            y=stats_df["Error mean (nm)"],
            yerr=stats_df["Error SD (nm)"], fmt="none", c="k")

plt.gca().set_ylim(bottom=0)